In [1]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score as CVS, train_test_split as TTS
from sklearn.metrics import mean_squared_error as MSE
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import datetime
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
accident=pd.read_csv('D:/Thesis/UK/Thanasis/Accident_Information.csv')

C:\Users\admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
vehicle=pd.read_csv('D:/Thesis/UK/Thanasis/Vehicle_Information.csv',encoding='ISO-8859-1')

In [4]:
pd.set_option('display.max_columns', 1000)

# Browse
## accident

In [5]:
accident.head()

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Control,Junction_Detail,Latitude,Light_Conditions,Local_Authority_(District),Local_Authority_(Highway),Location_Easting_OSGR,Location_Northing_OSGR,Longitude,LSOA_of_Accident_Location,Number_of_Casualties,Number_of_Vehicles,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Police_Force,Road_Surface_Conditions,Road_Type,Special_Conditions_at_Site,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Year,InScotland
0,200501BS00001,A,3218.0,NaN,0.0,Serious,None,2005-01-04,Tuesday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.489096,Daylight,Kensington and Chelsea,Kensington and Chelsea,525680.0,178240.0,-0.191170,E01002849,1,1,0.0,1.0,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,17:42,Urban,Raining no high winds,2005,No
1,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,Auto traffic signal,Crossroads,51.520075,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524170.0,181650.0,-0.211708,E01002909,1,1,0.0,5.0,Metropolitan Police,Dry,Dual carriageway,None,30.0,17:36,Urban,Fine no high winds,2005,No
2,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.525301,Darkness - lights lit,Kensington and Chelsea,Kensington and Chelsea,524520.0,182240.0,-0.206458,E01002857,1,2,0.0,0.0,Metropolitan Police,Dry,Single carriageway,None,30.0,00:15,Urban,Fine no high winds,2005,No
3,200501BS00004,A,3220.0,NaN,0.0,Slight,None,2005-01-07,Friday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.482442,Daylight,Kensington and Chelsea,Kensington and Chelsea,526900.0,177530.0,-0.173862,E01002840,1,1,0.0,0.0,Metropolitan Police,Dry,Single carriageway,None,30.0,10:35,Urban,Fine no high winds,2005,No
4,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-10,Monday,1.0,Data missing or out of range,Not at junction or within 20 metres,51.495752,Darkness - lighting unknown,Kensington and Chelsea,Kensington and Chelsea,528060.0,179040.0,-0.156618,E01002863,1,1,0.0,0.0,Metropolitan Police,Wet or damp,Single carriageway,None,30.0,21:13,Urban,Fine no high winds,2005,No


In [6]:
print('accident:',accident.shape)
print('vehicle:',vehicle.shape)

accident: (2047256, 34)
vehicle: (2177205, 24)


In [7]:
accident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047256 entries, 0 to 2047255
Data columns (total 34 columns):
 #   Column                                       Dtype  
---  ------                                       -----  
 0   Accident_Index                               object 
 1   1st_Road_Class                               object 
 2   1st_Road_Number                              float64
 3   2nd_Road_Class                               object 
 4   2nd_Road_Number                              float64
 5   Accident_Severity                            object 
 6   Carriageway_Hazards                          object 
 7   Date                                         object 
 8   Day_of_Week                                  object 
 9   Did_Police_Officer_Attend_Scene_of_Accident  float64
 10  Junction_Control                             object 
 11  Junction_Detail                              object 
 12  Latitude                                     float64
 13  Light_Condit

In [8]:
missing = accident.isnull().sum().reset_index()
missing.columns = ['Feature', 'Missing_Count']
missing['Missing_Percent(%)'] = missing['Missing_Count'].apply(lambda x: x / accident.shape[0] * 100)
missing.loc[missing['Missing_Count']>0,:]

,Feature,Missing_Count,Missing_Percent(%)
2,1st_Road_Number,2,0.000098
3,2nd_Road_Class,844272,41.239200
4,2nd_Road_Number,17593,0.859345
9,Did_Police_Officer_Attend_Scene_of_Accident,278,0.013579
12,Latitude,174,0.008499
16,Location_Easting_OSGR,164,0.008011
17,Location_Northing_OSGR,164,0.008011
18,Longitude,175,0.008548
19,LSOA_of_Accident_Location,144953,7.080355
22,Pedestrian_Crossing-Human_Control,2920,0.142630


In [9]:
from collections import Counter
pass_list=['Accident_Index','Date','Local_Authority_(District)','Local_Authority_(Highway)','LSOA_of_Accident_Location','Time']
for i in accident.columns.tolist():
    if accident[i].dtypes == object and i not in pass_list:
        print(i, Counter(accident[i]),'\n')
    else:
        continue
# give way: 让行标志 Authorised person: 猜测为协勤人员
#slip road: 匝道

1st_Road_Class Counter({'A': 926729, 'Unclassified': 603938, 'B': 258076, 'C': 174953, 'Motorway': 78071, 'A(M)': 5489}) 

2nd_Road_Class Counter({nan: 844272, 'Unclassified': 814983, 'A': 201317, 'C': 92074, 'B': 79657, 'Motorway': 13428, 'A(M)': 1525}) 

Accident_Severity Counter({'Slight': 1734548, 'Serious': 286339, 'Fatal': 26369}) 

Carriageway_Hazards Counter({'None': 2007807, 'Other object on road': 16111, 'Any animal in carriageway (except ridden horse)': 10416, 'Pedestrian in carriageway - not injured': 4684, 'Previous accident': 3105, 'Data missing or out of range': 2746, 'Vehicle load on road': 2387}) 

Day_of_Week Counter({'Friday': 335183, 'Wednesday': 308580, 'Thursday': 308240, 'Tuesday': 306292, 'Monday': 290482, 'Saturday': 273152, 'Sunday': 225327}) 

Junction_Control Counter({'Give way or uncontrolled': 988313, 'Data missing or out of range': 754311, 'Auto traffic signal': 211335, 'Not at junction or within 20 metres': 77304, 'Stop sign': 12333, 'Authorised person':

In [10]:
Counter(accident['Number_of_Casualties'])

Counter({1: 1576667,
         2: 324486,
         5: 12306,
         4: 33337,
         3: 92664,
         8: 674,
         7: 1567,
         6: 4575,
         10: 190,
         9: 329,
         23: 5,
         17: 20,
         13: 50,
         11: 114,
         19: 15,
         15: 25,
         16: 22,
         12: 65,
         14: 37,
         18: 12,
         20: 4,
         29: 6,
         21: 11,
         35: 2,
         22: 11,
         26: 8,
         45: 2,
         41: 2,
         36: 4,
         40: 2,
         25: 4,
         28: 2,
         42: 4,
         68: 1,
         27: 6,
         32: 1,
         62: 1,
         47: 1,
         48: 1,
         24: 5,
         43: 2,
         87: 2,
         63: 1,
         51: 2,
         33: 2,
         38: 2,
         46: 1,
         70: 1,
         93: 1,
         54: 1,
         34: 2,
         58: 1})

In [11]:
Counter(accident['Number_of_Vehicles'])

Counter({1: 617572,
         2: 1219250,
         4: 35067,
         3: 163519,
         5: 7813,
         6: 2393,
         8: 396,
         7: 864,
         13: 14,
         10: 90,
         11: 37,
         18: 3,
         9: 177,
         20: 2,
         17: 1,
         15: 4,
         12: 21,
         14: 14,
         22: 1,
         16: 8,
         28: 1,
         32: 1,
         19: 2,
         29: 1,
         34: 1,
         67: 1,
         21: 1,
         37: 1,
         23: 1})

In [12]:
Counter(accident['Pedestrian_Crossing-Human_Control'])# {0,1,2,nan}

Counter({0.0: 2031163,
         2.0: 8123,
         1.0: 5050,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
  

In [13]:
Counter(accident['Pedestrian_Crossing-Physical_Facilities'])# {0,1,4,5,7,8,nan}

Counter({1.0: 55785,
         5.0: 138133,
         0.0: 1695605,
         8.0: 39135,
         4.0: 109447,
         7.0: 5591,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
    

In [14]:
Counter(accident['Speed_limit'])# {10,15,20,30,40,50,60,70}

Counter({30.0: 1306174,
         40.0: 168357,
         50.0: 69479,
         20.0: 38399,
         60.0: 317469,
         70.0: 147305,
         10.0: 19,
         15.0: 16,
         0.0: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1,
         nan: 1})

## vehicle

In [15]:
vehicle.head()

,Accident_Index,Age_Band_of_Driver,Age_of_Vehicle,Driver_Home_Area_Type,Driver_IMD_Decile,Engine_Capacity_.CC.,Hit_Object_in_Carriageway,Hit_Object_off_Carriageway,Journey_Purpose_of_Driver,Junction_Location,make,model,Propulsion_Code,Sex_of_Driver,Skidding_and_Overturning,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact,Year
0,200401BS00001,26 - 35,3.0,Urban area,4.0,1588.0,None,None,Data missing or out of range,Data missing or out of range,ROVER,45 CLASSIC 16V,Petrol,Male,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,2,109,Data missing or out of range,Front,2004
1,200401BS00002,26 - 35,NaN,Urban area,3.0,NaN,None,None,Data missing or out of range,Data missing or out of range,BMW,C1,NaN,Male,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,109,Data missing or out of range,Front,2004
2,200401BS00003,26 - 35,4.0,Data missing or out of range,NaN,998.0,None,None,Data missing or out of range,Data missing or out of range,NISSAN,MICRA CELEBRATION 16V,Petrol,Male,None,No tow/articulation,Did not leave carriageway,0.0,Turning right,1,109,Data missing or out of range,Front,2004
3,200401BS00003,66 - 75,NaN,Data missing or out of range,NaN,NaN,None,None,Data missing or out of range,Data missing or out of range,LONDON TAXIS INT,TXII GOLD AUTO,NaN,Male,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,2,109,Data missing or out of range,Front,2004
4,200401BS00004,26 - 35,1.0,Urban area,4.0,124.0,None,None,Data missing or out of range,Data missing or out of range,PIAGGIO,VESPA ET4,Petrol,Male,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,Data missing or out of range,Front,2004


In [16]:
vehicle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2177205 entries, 0 to 2177204
Data columns (total 24 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   Accident_Index                    object 
 1   Age_Band_of_Driver                object 
 2   Age_of_Vehicle                    float64
 3   Driver_Home_Area_Type             object 
 4   Driver_IMD_Decile                 float64
 5   Engine_Capacity_.CC.              float64
 6   Hit_Object_in_Carriageway         object 
 7   Hit_Object_off_Carriageway        object 
 8   Journey_Purpose_of_Driver         object 
 9   Junction_Location                 object 
 10  make                              object 
 11  model                             object 
 12  Propulsion_Code                   object 
 13  Sex_of_Driver                     object 
 14  Skidding_and_Overturning          object 
 15  Towing_and_Articulation           object 
 16  Vehicle_Leaving_Carriageway       ob

In [17]:
from collections import Counter
pass_list=['Accident_Index','make','model']#汽车品牌，车型
for i in vehicle.columns.tolist():
    if vehicle[i].dtypes == object and i not in pass_list:
        print(i, Counter(vehicle[i]),'\n')
    else:
        continue

Age_Band_of_Driver Counter({'26 - 35': 450531, '36 - 45': 435686, '46 - 55': 348762, '21 - 25': 238765, '56 - 65': 206181, '16 - 20': 175874, 'Data missing or out of range': 171052, '66 - 75': 91454, 'Over 75': 54236, '11 - 15': 3655, '6 - 10': 884, '0 - 5': 125}) 

Driver_Home_Area_Type Counter({'Urban area': 1436598, 'Data missing or out of range': 334344, 'Rural': 232360, 'Small town': 173903}) 

Hit_Object_in_Carriageway Counter({'None': 2087824, 'Kerb': 34354, 'Parked vehicle': 27401, 'Bollard or refuge': 9981, 'Other object': 5715, 'Any animal (except ridden horse)': 2683, 'Central island of roundabout': 2190, 'Open door of vehicle': 1919, 'Bridge (side)': 1441, 'Data missing or out of range': 1302, 'Previous accident': 1023, 'Road works': 957, 'Bridge (roof)': 415}) 

Hit_Object_off_Carriageway Counter({'None': 1989741, 'Other permanent object': 56640, 'Tree': 29161, 'Entered ditch': 18301, 'Road sign or traffic signal': 17394, 'Central crash barrier': 16538, 'Near/Offside crash

In [22]:
vehicle.groupby(['Age_of_Vehicle'])['Accident_Index'].count()

Age_of_Vehicle
1.0      180333
2.0      161072
3.0      148665
4.0      144493
5.0      138464
          ...  
91.0          2
95.0          1
104.0         1
105.0         1
111.0         1
Name: Accident_Index, Length: 88, dtype: int64

In [19]:
vehicle.make.nunique()

535

In [20]:
vehicle.model.nunique()

35723

# Preprocessing
## accident

In [ ]:
data=data.drop(['Duration','Minute'],axis=1)

In [ ]:
pd.set_option('display.max_columns', 1000)
data

In [ ]:
data = data.replace([True, False], [1,0])
data = data.replace(['Day','Night'],[1,0])

In [ ]:
data

In [ ]:
# dataset为数据集  product_tags为需要编码的特征列(假设为第一列)
product_tags = ['Side','Timezone','Wind_Direction']
from sklearn.preprocessing import LabelEncoder
for col in product_tags:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.transform(data[col])

In [ ]:
data

In [ ]:
le.classes_  # 查看有哪些标签

In [ ]:
X=data.drop(['Severity4','Severity'],axis=1)
y=data.Severity4

In [ ]:
X

In [ ]:
y[30]

In [ ]:
X_train, X_test, y_train, y_test = TTS(X, y, test_size=0.30, random_state=710)

In [ ]:

# 转换为Dataset数据格式
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'binary',  # 目标函数
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 31,  # 叶子节点数
    'learning_rate': 0.05,  # 学习速率
    'feature_fraction': 0.9,  # 建树的特征选择比例
    'bagging_fraction': 0.8,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}

# 模型训练
gbm = lgb.train(params, lgb_train, num_boost_round=10000, valid_sets=lgb_eval, early_stopping_rounds=5)

# 模型预测
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

y_pred_bi=y_pred.copy()
y_pred_bi[y_pred>0.5]=1
y_pred_bi[y_pred_bi!=1]=0
# 模型评估
print('The rmse of prediction is:', MSE(y_test, y_pred) ** 0.5)
# cv_results = lgb.cv(params, data_train, num_boost_round=1000, nfold=5, is_unbalance=True, stratified=False, shuffle=True, metrics='auc',early_stopping_rounds=5,seed=0)
# print('best n_estimators:', len(cv_results['auc-mean']))
# print('best cv score:', pd.Series(cv_results['auc-mean']).max())

In [ ]:
# # 模型训练
# gbm = LGBMClassifier(objective= 'binary', num_leaves=31, learning_rate=0.05, n_estimators=100)
# gbm.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l1', early_stopping_rounds=5,categorical_feature=['Side', 
#                          'Timezone','Wind_Direction'],is_unbalance=True)

# # 模型存储
# # joblib.dump(gbm, 'loan_model.pkl')
# # # 模型加载
# # gbm = joblib.load('loan_model.pkl')

# # 模型预测
# y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)

# # 模型评估
# print('The rmse of prediction is:', MSE(y_test, y_pred) ** 0.5)

# # 特征重要度
# print('Feature importances:', list(gbm.feature_importances_))

# # 网格搜索，参数优化
# estimator = LGBMClassifier(num_leaves=31)
# param_grid = {
#     'learning_rate': [0.01, 0.1, 1],
#     'n_estimators': [100, 110,120]
# }
# gbm = GridSearchCV(estimator, param_grid)
# gbm.fit(X_train, y_train)
# print('Best parameters found by grid search are:', gbm.best_params_)


In [ ]:
# accuracy_train = gbm.score(X_train, y_train)
# print("Train Accuracy: %.1f%%"% (accuracy_train*100))
# accuracy_test = gbm.score(X_test,y_test)
# print("Test Accuracy: %.1f%%"% (accuracy_test*100))

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test,y_pred_bi))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
sns.set(font_scale = 1)
confmat = multilabel_confusion_matrix(y_true=y_test, y_pred=y_pred_bi,
                           labels=[1])

conf_matrix = pd.DataFrame(data=confmat[0],
                           columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,10))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu").set_title(
    "Confusion Matrix (Resampled Data)\n-Default LightGBM", fontsize=16)
plt.show()

In [ ]:
# importances=dict(zip(gbm.feature_names,gbm.feature_importance()))
# importances

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def plotImp(model, X , num = 20):
    feature_imp = pd.DataFrame({'Value':model.feature_importance(),'Feature':X.columns})
    plt.figure(figsize=(40, 20))
    sns.set(font_scale = 3)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                        ascending=False)[0:num])
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')
    plt.show()
plotImp(gbm, X=X , num = 20)